In [ ]:
import numpy as np
import scipy
import sympy

In [ ]:
class GF:
    p: int
    value: int

    def __init__(self, value: int, p: int = 2) -> None:
        self.value = value
        self.p = p

    def __add__(self, other):
        if not isinstance(other, GF):
            return self + GF(other, self.p)

        return GF((self.value + other.value) % self.p, self.p)

    def __sub__(self, other):
        if not isinstance(other, GF):
            return self - GF(other, self.p)

        return GF((self.value - other.value) % self.p, self.p)

    def __inv__(self):
        return GF((-self.value) % self.p, self.p)

    def __mul__(self, other):
        if not isinstance(other, GF):
            return self * GF(other, self.p)

        return GF(int(self.value * other.value) % self.p, self.p)

    def __truediv__(self, other):
        if not isinstance(other, GF):
            return self / GF(other, self.p)

        # find the inverse of other
        for value in range(self.p):
            if (value * other.value) % self.p == 1:
                return self * GF(value, self.p)

        raise Exception("cannot divide")

    def __str__(self) -> str:
        return f"{self.value} (mod {self.p})"


In [ ]:
# linear codes


class LinearCode:
    # dim = n-k x n
    H: np.ndarray[np.float64]
    # dim = n x k
    G: np.ndarray[np.float64]

    p: int

    def __init__(self, check: np.ndarray[np.float64], p: int = 2):
        self.H = check
        self.p = p

        # compute basis
        H = sympy.Matrix(self.H)
        # thank you sympy for the symbolic nullspace, reshape to k x n before transposing
        ns = np.transpose(np.array(H.nullspace()).reshape((-1, self.H.shape[1])))
        self.G = ns % self.p

    def size(self):
        # n x k
        return self.G.shape

    def info_bits(self):
        n, k = self.size()

        infos = []
        for index in range(n):
            if np.count_nonzero(self.G[index]) == 1:
                infos.append(index)

        return np.array(infos)

    def check_bits(self):
        infos = self.info_bits()
        n, k = self.size()

        return np.array(filter(lambda i: i not in infos, range(n)))

    def encode(self, word: np.ndarray[np.float64]):
        return np.array((self.G @ word) % self.p, dtype="int")

    def decode(self, codeword: np.ndarray[np.float64]):
        remainder = np.mod(self.H @ codeword, self.p)

        # all zeros
        if not np.any(remainder):
            # decode
            return codeword[self.info_bits()]

        # try to find a linear combination from the column space
        # this part is scuffed, it will only work for single binary errors
        x, _, r, _ = np.linalg.lstsq(self.H, remainder, rcond=None)
        highest = x.argmax()
        codeword[highest] = 1 if codeword[highest] == 0 else 0
        return codeword[self.info_bits()]

    def min_distance(self):
        # idk how to compute this except brute force
        # i am not sure if the rank H method works
        return np.linalg.matrix_rank(self.H)

    def max_detection(self):
        return self.min_distance() - 1

    def max_correction(self):
        return (self.min_distance() - 1) // 2


lc = LinearCode(
    np.array([[0, 0, 0, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 1]])
)

test = np.array([1, 0, 1, 1])
print(f"{test=}")
encoded = lc.encode(test)
print(f"{encoded=}")
encoded[1] = 0
print(f"errored={encoded}")
decoded = lc.decode(encoded)
print(f"{decoded=}")

print(lc.min_distance())
print(lc.max_detection())
print(lc.max_correction())
